In [1]:
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import backend as K
from keras import regularizers
import math

2022-11-21 06:53:17.999618: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
data = np.load("./images.npy")
label = np.load("./labels.npy")
data = data / 255.

In [3]:
ya_hour = 2 * np.pi * (label[:, 0] + label[:, 1] / 60) / 12
ya_hour1 = np.sin(ya_hour)
ya_hour2 = np.cos(ya_hour)
y_l = np.stack((ya_hour1, ya_hour2), axis=1)
y_l = np.concatenate((y_l, label), axis=1)

In [4]:
X, X_test, y, y_test = train_test_split(data, y_l, test_size=0.2, random_state=42)
print(f"Train set: {X.shape}")
print(f"Test set: {X_test.shape}")
y = y[:, :2]
y_test = y_test[:, 2:]

Train set: (14400, 150, 150)
Test set: (3600, 150, 150)


In [5]:
y_test = y_test[:, 0] + y_test[:, 1] / 60

In [9]:
K.clear_session()
final_model = keras.Sequential(
    [
        keras.Input(shape=(150,150,1)),
        layers.Conv2D(16, kernel_size=(3, 3), activation=keras.layers.LeakyReLU(alpha=0.01),strides=1,kernel_regularizer=regularizers.L2(1e-4)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(32, kernel_size=(3, 3), activation=keras.layers.LeakyReLU(alpha=0.01),strides=2,kernel_regularizer=regularizers.L2(1e-4)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation=keras.layers.LeakyReLU(alpha=0.01),strides=1,kernel_regularizer=regularizers.L2(1e-4)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dropout(0.1),
        layers.Dense(64,activation=keras.layers.LeakyReLU(alpha=0.01),kernel_regularizer=regularizers.L2(1e-4)),
        layers.Normalization(axis=None),
        layers.Dense(2,activation="tanh"),
    ]
)

final_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 148, 148, 16)      160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 74, 74, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 36, 36, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 18, 18, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 16, 16, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 8, 8, 64)         0

In [10]:
batch_size = 128
epochs = 200
final_model.compile(loss="mse", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001,), metrics="mse")

In [ ]:
checkpoint_filepath = "./best_model.hdf5"
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_mse',
    mode='min',
    save_best_only=True
)
final_model.fit(X, y, batch_size=batch_size, epochs=epochs, validation_split=0.1, callbacks=[model_checkpoint_callback])

Epoch 1/200
102/102 [==============================] - 2s 19ms/step - loss: 0.5097 - mse: 0.5003 - val_loss: 0.5039 - val_mse: 0.4987
Epoch 2/200
102/102 [==============================] - 1s 14ms/step - loss: 0.4463 - mse: 0.4400 - val_loss: 0.3511 - val_mse: 0.3421
Epoch 3/200
102/102 [==============================] - 1s 13ms/step - loss: 0.3175 - mse: 0.3062 - val_loss: 0.2672 - val_mse: 0.2535
Epoch 4/200
102/102 [==============================] - 1s 13ms/step - loss: 0.2239 - mse: 0.2080 - val_loss: 0.1850 - val_mse: 0.1670
Epoch 5/200
102/102 [==============================] - 1s 13ms/step - loss: 0.1608 - mse: 0.1412 - val_loss: 0.1376 - val_mse: 0.1168
Epoch 6/200
102/102 [==============================] - 2s 18ms/step - loss: 0.1275 - mse: 0.1056 - val_loss: 0.1165 - val_mse: 0.0938
Epoch 7/200
102/102 [==============================] - 2s 15ms/step - loss: 0.1009 - mse: 0.0777 - val_loss: 0.0961 - val_mse: 0.0725
Epoch 8/200
102/102 [==============================] - 2s 15ms

In [ ]:
final_model.load_weights(checkpoint_filepath)
output=final_model.predict(X_test)
predict1=output[:,0]/np.sqrt(np.square(output[:,0])+np.square(output[:,1]))
predict2=output[:,1]/np.sqrt(np.square(output[:,0])+np.square(output[:,1]))

In [ ]:
angle=[]
for i in range(len(predict1)):
    a_acos = math.acos(predict2[i])
    if predict1[i] < 0:
        angle.append( math.degrees(-a_acos) % 360 )
    else: 
        angle.append( math.degrees(a_acos) )

In [ ]:
angle=np.stack(angle)
time_p=12*angle/360

def common_sense_reg(y_true, y_pred):
    d = np.abs(y_true-y_pred)
    return (1-(d//6))*(d)+(d//6)*(12-d)

common_sense_reg(y_test,time_p).mean()